In [2]:
import  pandas as pd
# Load key_attributes.csv file
key_attributes_df = pd.read_csv('census_starter.csv')
# Unpivot the percentage of broadband access columns
# pct_bb_cols = [col for col in key_attributes_df.columns if col.startswith('pct_')]
# key_attributes_df = key_attributes_df.melt(id_vars=['cfips'], value_vars=pct_bb_cols, var_name='pct_categories', value_name='pct_actual')
key_attributes_df.describe()

# extract the year from the column names in the other_df DataFrame
key_attributes_df['year'] = key_attributes_df.columns.str.extract('(\d{4})')

key_attributes_df = key_attributes_df.melt(id_vars=['cfips', 'year'], var_name='attribute', value_name='pct_value')

key_attributes_df[["extra","real_year"]]=key_attributes_df["attribute"].str.split("_2",expand=True)
key_attributes_df.drop("extra",axis=1,inplace=True)

key_attributes_df["real_year"]='2' + key_attributes_df["real_year"].str.zfill(2)

# drop all rows where col1 and col2 don't match
key_attributes_df = key_attributes_df.drop(index=key_attributes_df[key_attributes_df['real_year'] != key_attributes_df['year']].index)

key_attributes_df["attribute"]=key_attributes_df["attribute"].str[:-5]
print(key_attributes_df)


       cfips  year      attribute  pct_value real_year
0       1001  2017         pct_bb       76.6      2017
6       1013  2017         pct_bb       58.2      2017
11      1023  2017         pct_bb       52.3      2017
16      1033  2017         pct_bb       65.9      2017
21      1043  2017         pct_bb       65.2      2017
...      ...   ...            ...        ...       ...
75412   1009  2021  median_hh_inc    52830.0      2021
75418   1021  2021  median_hh_inc    56243.0      2021
75423   1031  2021  median_hh_inc    59034.0      2021
75428   1041  2021  median_hh_inc    43103.0      2021
75433   1051  2021  median_hh_inc    67597.0      2021

[125 rows x 5 columns]


In [3]:
test_df=pd.read_csv('train.csv')
test_df[["year","month","date"]]=test_df["first_day_of_month"].str.split("-",expand=True)


merged_df = pd.merge(test_df, key_attributes_df, on=['cfips', 'year'])
print(merged_df)

merged_df.to_csv('merged_census_test.csv', index=False)

# pivot the DataFrame
merged_df = merged_df.pivot(index=['row_id','cfips','county','state','first_day_of_month','microbusiness_density','active','year','month','date','real_year'], columns='attribute', values='pct_value').reset_index()


# print the resulting DataFrame
print(merged_df)

merged_df.to_csv('merged_census_test.csv', index=False)

              row_id  cfips          county    state first_day_of_month  \
0    1005_2019-08-01   1005  Barbour County  Alabama         2019-08-01   
1    1005_2019-08-01   1005  Barbour County  Alabama         2019-08-01   
2    1005_2019-08-01   1005  Barbour County  Alabama         2019-08-01   
3    1005_2019-08-01   1005  Barbour County  Alabama         2019-08-01   
4    1005_2019-08-01   1005  Barbour County  Alabama         2019-08-01   
..               ...    ...             ...      ...                ...   
720  1051_2021-12-01   1051   Elmore County  Alabama         2021-12-01   
721  1051_2021-12-01   1051   Elmore County  Alabama         2021-12-01   
722  1051_2021-12-01   1051   Elmore County  Alabama         2021-12-01   
723  1051_2021-12-01   1051   Elmore County  Alabama         2021-12-01   
724  1051_2021-12-01   1051   Elmore County  Alabama         2021-12-01   

     microbusiness_density  active  year month date         attribute  \
0                 1.073138

In [4]:
y=merged_df.microbusiness_density
merged_df = pd.get_dummies(merged_df, columns=['row_id'])

# print(merged_df)
census_features = ["row_id","cfips"]

X=merged_df.drop(["microbusiness_density","county","first_day_of_month","state","active","year","month","date","real_year","median_hh_inc","pct_bb","pct_college","pct_foreign_born","pct_it_workers"],axis=1)

print(X)

from sklearn.model_selection import train_test_split


train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)



     cfips  row_id_1005_2019-08-01  row_id_1005_2019-09-01  \
0     1005                       1                       0   
1     1005                       0                       1   
2     1005                       0                       0   
3     1005                       0                       0   
4     1005                       0                       0   
..     ...                     ...                     ...   
140   1051                       0                       0   
141   1051                       0                       0   
142   1051                       0                       0   
143   1051                       0                       0   
144   1051                       0                       0   

     row_id_1005_2019-10-01  row_id_1005_2019-11-01  row_id_1005_2019-12-01  \
0                         0                       0                       0   
1                         0                       0                       0   
2                 

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(melb_preds)
rf_val_mae=mean_absolute_error(val_y, melb_preds)
print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))


[1.31665552 1.60886865 1.60886865 1.26695652 1.26695652 1.54476869
 1.35109268 1.60886865 0.93062023 1.60886865 1.16205463 1.16205463
 2.18536854 1.35109268 2.76474609 1.16205463 1.31665552 2.18536854
 1.54476869 1.31665552 2.07075607 2.18536854 2.18536854 2.07075607
 0.99526651 1.37787372 1.57246711 0.99526651 1.20385567 0.93062023
 1.20385567 1.60886865 1.16205463 1.57246711 1.02808789 1.35109268
 1.31665552]
Validation MAE for Random Forest Model: 0.08164449721621568


In [6]:
final_test_df=pd.read_csv('test.csv')
print(val_X)



     cfips  row_id_1005_2019-08-01  row_id_1005_2019-09-01  \
7     1007                       0                       0   
24    1009                       0                       0   
27    1009                       0                       0   
66    1029                       0                       0   
73    1029                       0                       0   
98    1039                       0                       0   
62    1027                       0                       0   
22    1009                       0                       0   
111   1041                       0                       0   
26    1009                       0                       0   
40    1019                       0                       0   
44    1019                       0                       0   
129   1049                       0                       0   
60    1027                       0                       0   
136   1051                       0                       0   
37    10

In [7]:
test_df = pd.get_dummies(test_df, columns=['row_id'])

X_test = test_df.drop("first_day_of_month",axis=1)

print(X_test)

y_pred=forest_model.predict(X_test)

print(y_pred)